In [ ]:
#!/usr/bin/env python3
# ------------------------------------------------------------
# Generate 1D SCM wind forcing (360 stations) matching 3D 5-km y-resolution.
# Files:
#   Wind_5mps_001.nc ... Wind_5mps_360.nc
# Each file: Time x Lat(2) x Lon(2) with Taux, Tauy, u10, v10 (uniform 2x2 patch)
# ------------------------------------------------------------
import os
import numpy as np
from netCDF4 import Dataset

# ============================
# USER CONFIG
# ============================
OUT_DIR = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/forcing_1D_5mps_360_wind"
os.makedirs(OUT_DIR, exist_ok=True)

DT_SEC   = 600
N_DAYS   = 3
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# --- hurricane physics (keep your original) ---
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0

# ===== choose speed case =====
SPEED_MPS = 5.0
X0 = 3.432e6
Y0 = 9.0e5
transdir = np.deg2rad(180.0)
f  = 5.5659e-5
t0_sec = 0.0

# inflow angle params (keep your original)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

# --- station line (physical) ---
# IMPORTANT: pick a tracer-center x, away from boundary
X_REF_KM = 2947.5          # 推荐：离边界10格，且正好落在 2.5+5*k
x_ref_m  = X_REF_KM * 1000.0

# 360 y-centers: 2.5, 7.5, ..., 1797.5 km  (match 3D tracer centers)
y_km_list = 2.5 + 5.0*np.arange(360)
y_m_list  = y_km_list * 1000.0

# dummy Lon/Lat for 2x2 patch (match your SCM fixed hgrid)
Lon_1d = np.array([-1.0, 1.0], dtype=np.float64)
Lat_1d = np.array([-1.0, 1.0], dtype=np.float64)

# ============================
# HELPERS (same physics)
# ============================
def Cd_from_du10(du10):
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

def stress_point(tsec, x_m, y_m):
    dp = p_n - p_c
    B  = (Umax**2) * rho_a * np.e / dp

    XC = X0 + (tsec - t0_sec) * SPEED_MPS * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * SPEED_MPS * np.sin(transdir)

    dX = x_m - XC
    dY = y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    U_TS = 0.5*SPEED_MPS*np.cos(transdir)
    V_TS = 0.5*SPEED_MPS*np.sin(transdir)

    rrB_e  = rad_edge**(-B)
    tmpA_e = (rrB_e*B) * dp
    r_e    = rad_edge * Rmax
    tmpB_e = (0.5*r_e*f) * rho_a
    num_e  = tmpA_e * np.exp(-rrB_e)
    den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
    U10_edge = float(num_e / max(den_e, 1e-20))

    U10 = 0.0
    if (rr > 1e-3) and (rr <= rad_edge):
        rrB = rr**(-B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10  = float(num / max(den, 1e-20))
    elif (rr > rad_edge) and (rr < rad_ambient):
        fac = (rad_ambient - rr)/(rad_ambient - rad_edge)
        U10 = float(fac * U10_edge)

    Adir = np.arctan2(dY, dX)
    RSTR = min(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*SPEED_MPS) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*SPEED_MPS) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if rr > rad_edge:
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph *= taper

    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    Cd = float(Cd_from_du10(du10))
    Taux = rho_a * Cd * du10 * u10x
    Tauy = rho_a * Cd * du10 * u10y
    return float(Taux), float(Tauy), float(u10x), float(u10y)

# ============================
# TIME AXIS (+tail)
# ============================
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")
t_min = (t_sec // 60).astype("int32")
records_per_day = int(round(86400/DT_SEC))
Nt = t_sec.size

FILL = np.float32(9.96921e36)

print(f"[INFO] writing 360 wind files, Nt={Nt}, dt={DT_SEC}s, X_REF_KM={X_REF_KM}")

# ============================
# WRITE FILES
# ============================
for sid, y_m in enumerate(y_m_list, start=1):
    out_path = os.path.join(OUT_DIR, f"Wind_{int(SPEED_MPS)}mps_{sid:03d}.nc")

    # compute time series first (faster I/O)
    taux_ts = np.empty(Nt, dtype="f4")
    tauy_ts = np.empty(Nt, dtype="f4")
    u10_ts  = np.empty(Nt, dtype="f4")
    v10_ts  = np.empty(Nt, dtype="f4")

    for k, tt in enumerate(t_sec):
        taux, tauy, u10, v10 = stress_point(int(tt), x_ref_m, float(y_m))
        taux_ts[k] = taux
        tauy_ts[k] = tauy
        u10_ts[k]  = u10
        v10_ts[k]  = v10

    nc = Dataset(out_path, "w", format="NETCDF4_CLASSIC")
    nc.createDimension("Time", None)
    nc.createDimension("Lat", 2)
    nc.createDimension("Lon", 2)

    vT = nc.createVariable("Time", "i4", ("Time",))
    vT.units = f"minutes since {START_TIME_STR}"
    vT.calendar = CALENDAR
    vT[:] = t_min

    vLat = nc.createVariable("Lat", "f8", ("Lat",))
    vLon = nc.createVariable("Lon", "f8", ("Lon",))
    vLat.units = "degrees_north"; vLon.units = "degrees_east"
    vLat[:] = Lat_1d
    vLon[:] = Lon_1d

    kw = dict(fill_value=FILL, zlib=True, complevel=2, chunksizes=(1,2,2))
    vtx = nc.createVariable("Taux", "f4", ("Time","Lat","Lon"), **kw)
    vty = nc.createVariable("Tauy", "f4", ("Time","Lat","Lon"), **kw)
    vu  = nc.createVariable("u10",  "f4", ("Time","Lat","Lon"), **kw)
    vv  = nc.createVariable("v10",  "f4", ("Time","Lat","Lon"), **kw)

    # broadcast to 2x2 patch in ONE write each
    vtx[:, :, :] = taux_ts[:, None, None]
    vty[:, :, :] = tauy_ts[:, None, None]
    vu[:,  :, :] = u10_ts[:,  None, None]
    vv[:,  :, :] = v10_ts[:,  None, None]

    nc.close()

    if sid % 30 == 0 or sid in (1,360):
        print(f"[OK] {sid:03d}/360  y_km={y_km_list[sid-1]:.1f}  -> {out_path}")

print(f"[DONE] wrote 360 wind files into {OUT_DIR}")
print(f"[INFO] cadence hint: -{records_per_day}")


In [ ]:
#!/usr/bin/env python3
# ------------------------------------------------------------
#for 2mps wind 360 file generation
# Generate 1D SCM wind forcing (360 stations) matching 3D 5-km y-resolution.
# Files:
#   Wind_2mps_001.nc ... Wind_2mps_360.nc
# Each file: Time x Lat(2) x Lon(2) with Taux, Tauy, u10, v10 (uniform 2x2 patch)
# ------------------------------------------------------------
import os
import numpy as np
from netCDF4 import Dataset

# ============================
# USER CONFIG
# ============================
OUT_DIR = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/forcing_1D_2mps_360_wind"
os.makedirs(OUT_DIR, exist_ok=True)

DT_SEC   = 600
N_DAYS   = 3
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# --- hurricane physics (keep your original) ---
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0

# ===== choose speed case =====
SPEED_MPS = 2.0
X0 = 3.1728e6 
Y0 = 9.0e5
transdir = np.deg2rad(180.0)
f  = 5.5659e-5
t0_sec = 0.0

# inflow angle params (keep your original)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

# --- station line (physical) ---
# IMPORTANT: pick a tracer-center x, away from boundary
X_REF_KM = 2947.5          # 推荐：离边界10格，且正好落在 2.5+5*k
x_ref_m  = X_REF_KM * 1000.0

# 360 y-centers: 2.5, 7.5, ..., 1797.5 km  (match 3D tracer centers)
y_km_list = 2.5 + 5.0*np.arange(360)
y_m_list  = y_km_list * 1000.0

# dummy Lon/Lat for 2x2 patch (match your SCM fixed hgrid)
Lon_1d = np.array([-1.0, 1.0], dtype=np.float64)
Lat_1d = np.array([-1.0, 1.0], dtype=np.float64)

# ============================
# HELPERS (same physics)
# ============================
def Cd_from_du10(du10):
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

def stress_point(tsec, x_m, y_m):
    dp = p_n - p_c
    B  = (Umax**2) * rho_a * np.e / dp

    XC = X0 + (tsec - t0_sec) * SPEED_MPS * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * SPEED_MPS * np.sin(transdir)

    dX = x_m - XC
    dY = y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    U_TS = 0.5*SPEED_MPS*np.cos(transdir)
    V_TS = 0.5*SPEED_MPS*np.sin(transdir)

    rrB_e  = rad_edge**(-B)
    tmpA_e = (rrB_e*B) * dp
    r_e    = rad_edge * Rmax
    tmpB_e = (0.5*r_e*f) * rho_a
    num_e  = tmpA_e * np.exp(-rrB_e)
    den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
    U10_edge = float(num_e / max(den_e, 1e-20))

    U10 = 0.0
    if (rr > 1e-3) and (rr <= rad_edge):
        rrB = rr**(-B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10  = float(num / max(den, 1e-20))
    elif (rr > rad_edge) and (rr < rad_ambient):
        fac = (rad_ambient - rr)/(rad_ambient - rad_edge)
        U10 = float(fac * U10_edge)

    Adir = np.arctan2(dY, dX)
    RSTR = min(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*SPEED_MPS) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*SPEED_MPS) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if rr > rad_edge:
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph *= taper

    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    Cd = float(Cd_from_du10(du10))
    Taux = rho_a * Cd * du10 * u10x
    Tauy = rho_a * Cd * du10 * u10y
    return float(Taux), float(Tauy), float(u10x), float(u10y)

# ============================
# TIME AXIS (+tail)
# ============================
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")
t_min = (t_sec // 60).astype("int32")
records_per_day = int(round(86400/DT_SEC))
Nt = t_sec.size

FILL = np.float32(9.96921e36)

print(f"[INFO] writing 360 wind files, Nt={Nt}, dt={DT_SEC}s, X_REF_KM={X_REF_KM}")

# ============================
# WRITE FILES
# ============================
for sid, y_m in enumerate(y_m_list, start=1):
    out_path = os.path.join(OUT_DIR, f"Wind_{int(SPEED_MPS)}mps_{sid:03d}.nc")

    # compute time series first (faster I/O)
    taux_ts = np.empty(Nt, dtype="f4")
    tauy_ts = np.empty(Nt, dtype="f4")
    u10_ts  = np.empty(Nt, dtype="f4")
    v10_ts  = np.empty(Nt, dtype="f4")

    for k, tt in enumerate(t_sec):
        taux, tauy, u10, v10 = stress_point(int(tt), x_ref_m, float(y_m))
        taux_ts[k] = taux
        tauy_ts[k] = tauy
        u10_ts[k]  = u10
        v10_ts[k]  = v10

    nc = Dataset(out_path, "w", format="NETCDF4_CLASSIC")
    nc.createDimension("Time", None)
    nc.createDimension("Lat", 2)
    nc.createDimension("Lon", 2)

    vT = nc.createVariable("Time", "i4", ("Time",))
    vT.units = f"minutes since {START_TIME_STR}"
    vT.calendar = CALENDAR
    vT[:] = t_min

    vLat = nc.createVariable("Lat", "f8", ("Lat",))
    vLon = nc.createVariable("Lon", "f8", ("Lon",))
    vLat.units = "degrees_north"; vLon.units = "degrees_east"
    vLat[:] = Lat_1d
    vLon[:] = Lon_1d

    kw = dict(fill_value=FILL, zlib=True, complevel=2, chunksizes=(1,2,2))
    vtx = nc.createVariable("Taux", "f4", ("Time","Lat","Lon"), **kw)
    vty = nc.createVariable("Tauy", "f4", ("Time","Lat","Lon"), **kw)
    vu  = nc.createVariable("u10",  "f4", ("Time","Lat","Lon"), **kw)
    vv  = nc.createVariable("v10",  "f4", ("Time","Lat","Lon"), **kw)

    # broadcast to 2x2 patch in ONE write each
    vtx[:, :, :] = taux_ts[:, None, None]
    vty[:, :, :] = tauy_ts[:, None, None]
    vu[:,  :, :] = u10_ts[:,  None, None]
    vv[:,  :, :] = v10_ts[:,  None, None]

    nc.close()

    if sid % 30 == 0 or sid in (1,360):
        print(f"[OK] {sid:03d}/360  y_km={y_km_list[sid-1]:.1f}  -> {out_path}")

print(f"[DONE] wrote 360 wind files into {OUT_DIR}")
print(f"[INFO] cadence hint: -{records_per_day}")
